In [8]:
import pandas as pd
train_df = pd.read_csv('https://raw.githubusercontent.com/ralbu85/DataScience_2022S/master/final_project/Rain_Train.csv')
test_df = pd.read_csv('https://raw.githubusercontent.com/ralbu85/DataScience_2022S/master/final_project/Rain_Test.csv')

train_df = train_df.drop(columns = 'Unnamed: 0')
test_df = test_df.drop(columns = 'Unnamed: 0')

## data 생성

In [49]:
new_train_df = train_df.drop(columns='RainTomorrow')

merge_df = pd.concat([new_train_df, test_df])

merge_df['Month'] = merge_df['Date'].map(lambda x: x[5:7])
new_merge_X = merge_df.drop(columns='Date')

dummy_columns = ['WindGustDir', 'Location', 'WindDir9am', 'WindDir3pm', 'Month', 'RainToday']
new_merge_X = pd.get_dummies(new_merge_X,columns = dummy_columns, drop_first=True)

train_X = new_merge_X.iloc[:len(train_df)]
# y = pd.get_dummies(train_df['RainTomorrow'],columns = ['RainTomorrow'], drop_first=True)
y = train_df['RainTomorrow']


In [17]:
test_X = new_merge_X.iloc[len(train_df):]
test_X

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,...,Month_04,Month_05,Month_06,Month_07,Month_08,Month_09,Month_10,Month_11,Month_12,RainToday_Yes
0,15.5,39.6,0.0,56.0,9.0,9.0,45.0,12.0,1016.0,1012.4,...,0,0,0,0,0,0,0,0,0,0
1,17.1,38.3,0.0,48.0,20.0,20.0,35.0,19.0,1017.9,1012.3,...,0,0,0,0,0,0,0,0,0,0
2,26.0,33.1,0.0,41.0,7.0,7.0,46.0,37.0,1013.6,1012.1,...,0,0,0,0,0,0,0,0,0,0
3,19.0,35.2,0.0,33.0,7.0,9.0,60.0,34.0,1017.4,1014.7,...,0,0,0,0,0,0,0,0,0,0
4,20.5,36.1,0.0,39.0,9.0,13.0,54.0,25.0,1019.1,1016.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33908,3.5,21.8,0.0,31.0,15.0,13.0,59.0,27.0,1024.7,1021.2,...,0,0,1,0,0,0,0,0,0,0
33909,2.8,23.4,0.0,31.0,13.0,11.0,51.0,24.0,1024.6,1020.3,...,0,0,1,0,0,0,0,0,0,0
33910,3.6,25.3,0.0,22.0,13.0,9.0,56.0,21.0,1023.5,1019.1,...,0,0,1,0,0,0,0,0,0,0
33911,5.4,26.9,0.0,37.0,9.0,9.0,53.0,24.0,1021.0,1016.8,...,0,0,1,0,0,0,0,0,0,0


## GridSerachCV for Random Forest

In [37]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, cross_validate
from sklearn.ensemble import RandomForestClassifier

In [56]:
param_grid = { 
    'n_estimators': [300, 500, 700],
    'max_features': ['auto', 'sqrt'],
    'max_depth' : [45, 50, 55],
}

In [57]:
kfold=KFold(n_splits=5, shuffle=True)
clf = RandomForestClassifier()
grid=GridSearchCV(estimator=clf, param_grid=param_grid, scoring='roc_auc', cv=kfold, verbose=2, n_jobs=-1)

In [58]:
grid.fit(train_X, y.values.ravel())

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] END ..max_depth=45, max_features=auto, n_estimators=300; total time= 1.3min
[CV] END ..max_depth=45, max_features=auto, n_estimators=300; total time= 1.3min
[CV] END ..max_depth=45, max_features=auto, n_estimators=300; total time= 1.3min
[CV] END ..max_depth=45, max_features=auto, n_estimators=300; total time= 1.3min
[CV] END ..max_depth=45, max_features=auto, n_estimators=300; total time= 1.3min
[CV] END ..max_depth=45, max_features=auto, n_estimators=500; total time= 2.2min
[CV] END ..max_depth=45, max_features=auto, n_estimators=500; total time= 2.2min
[CV] END ..max_depth=45, max_features=auto, n_estimators=500; total time= 2.2min
[CV] END ..max_depth=45, max_features=auto, n_estimators=500; total time= 2.2min
[CV] END ..max_depth=45, max_features=auto, n_estimators=500; total time= 2.3min
[CV] END ..max_depth=45, max_features=sqrt, n_estimators=300; total time= 1.4min
[CV] END ..max_depth=45, max_features=sqrt, n_es

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
             estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [45, 50, 55],
                         'max_features': ['auto', 'sqrt'],
                         'n_estimators': [300, 500, 700]},
             scoring='roc_auc', verbose=2)

In [59]:
grid.best_score_

0.8915016918361636

In [60]:
grid.best_params_

{'max_depth': 50, 'max_features': 'auto', 'n_estimators': 700}

### 측정

In [61]:
grid.predict(test_X)

array(['No', 'No', 'No', ..., 'No', 'No', 'No'], dtype=object)